# 🖼️ 1. Datensatz vorbereiten

Jede Gruppe erhält einen Ordner mit **Rohbildern**. Diese sollen in [Makesense.ai](https://www.makesense.ai) annotiert werden.

## 🔧 Schritt 1: Annotation mit Makesense.ai
0. Ladet die Rohbilder mit eurer Gruppennummer unter folgender [URL](https://github.com/MrZinken/Hackathon-Bonn) runter
1. Öffnet [https://www.makesense.ai](https://www.makesense.ai)
2. Bilder hochladen
3. Modus: **Object Detection**
4. Klasse: z. B. `baum`, `überdachung`
5. Starten und rechts unten Polygon auswählen
6. Fleißig sein. Alle Objekte der Klasse müssen in dem Bild annotiert werden. Nicht vollständig annotierte Bilder verschlechtern die Performance signifikant.
7. Export: "Actions" -> "Export Annotations" -> "Single file in COCO JSON Format" 
8. Annotations File umbennen in "annotations.json" und zusammen mit den annotierten Bilder in den Ordner datensatz kopieren

👉 Jetzt bitte euren Ordner mit den annotierten Bildern hier per Drag and Drop hochladen.


In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import os
import json
from IPython.display import display

# Gruppennummer eintragen (Pflicht!)
gruppen_nummer = X  # <--- HIER EINTRAGEN!

# Check: Gruppennummer muss gesetzt und gültig sein
if not isinstance(gruppen_nummer, int) or gruppen_nummer <= 0:
    raise ValueError("❌ Bitte trage eine gültige Gruppennummer ein (z. B. gruppen_nummer = 61)")
else:
    print(f"👥 Gruppennummer erkannt: {gruppen_nummer}")


# Arbeitsverzeichnis dynamisch zusammensetzen
annotations_file = f"datensatz/annotations.json"
images_dir = f"datensatz"

# COCO laden mit Fehlerbehandlung
try:
    coco = COCO(annotations_file)
except Exception as e:
    print(f"❌ Fehler beim Laden der COCO-Datei: {e}")
    raise

# Bild- und Annotationen-Prüfung
missing_files = []
invalid_annotations = []

print("🔍 Prüfe Bilder und Annotationen...")
for img in coco.dataset["images"]:
    file_path = os.path.join(images_dir, img["file_name"])
    if not os.path.isfile(file_path):
        missing_files.append(img["file_name"])
    
    ann_ids = coco.getAnnIds(imgIds=img["id"])
    anns = coco.loadAnns(ann_ids)
    for ann in anns:
        if "category_id" not in ann:
            invalid_annotations.append({
                "image_file": img["file_name"],
                "annotation_id": ann.get("id", "unknown"),
                "annotation": ann
            })

# Fehlerberichte ausgeben
if missing_files:
    print(f"\n❌ Fehlende Bilddateien ({len(missing_files)}):")
    for f in missing_files:
        print(f" - {f}")
else:
    print("✅ Alle referenzierten Bilder sind vorhanden.")

if invalid_annotations:
    print(f"\n❌ Annotationen ohne gültige 'category_id' ({len(invalid_annotations)}):")
    for ann in invalid_annotations:
        print(f" - Bild: {ann['image_file']} | Annotation-ID: {ann['annotation_id']} → fehlt 'category_id'")
else:
    print("✅ Alle Annotationen enthalten eine gültige 'category_id'.")

# Falls kritische Fehler: abbrechen
if missing_files or invalid_annotations:
    print("\n⚠️ Bitte korrigiere die Fehler in deiner JSON-Datei, bevor du weitermachst.")
else:
    print("\n✅ Alles sieht gut aus. Starte Visualisierung...")

    # Erstes Bild anzeigen (optional)
    image_ids = coco.getImgIds()
    if image_ids:
        img_id = image_ids[0]
        img_info = coco.loadImgs(img_id)[0]
        img_path = os.path.join(images_dir, img_info['file_name'])

        # Bild laden
        image = Image.open(img_path)

        # Plot vorbereiten
        fig = plt.figure(figsize=(8, 8))
        plt.imshow(image)
        ax = plt.gca()

        # Annotations laden
        ann_ids = coco.getAnnIds(imgIds=img_id)
        anns = coco.loadAnns(ann_ids)

        # Maske zeichnen
        for ann in anns:
            if 'category_id' not in ann:
                continue  # Ungültige Annotation überspringen
            if 'segmentation' in ann and isinstance(ann['segmentation'], list):
                for seg in ann['segmentation']:
                    poly = np.array(seg).reshape((len(seg) // 2, 2))
                    patch = patches.Polygon(poly, fill=False, edgecolor='red', linewidth=2)
                    ax.add_patch(patch)

        plt.title(img_info['file_name'])
        plt.axis('off')
        plt.show()


: 

## 📈 Datensatz-Erweiterung durch Augmentation

Um die **Robustheit und Generalisierungsfähigkeit** des Modells zu verbessern, wurde der ursprüngliche Datensatz durch gezielte Bildaugmentierung erweitert. Dabei wurden für jedes bestehende Bild **bis zu zwei zusätzliche Varianten** erzeugt. Die Auswahl und Kombination der Augmentierungsschritte erfolgte **zufällig**, jedoch auf Basis definierter Wahrscheinlichkeiten und Begrenzungen.

### Eingesetzte Augmentierungsschritte

Die folgenden Transformationen wurden mit einer festgelegten Wahrscheinlichkeit pro Bild angewendet:

- **Horizontales Spiegeln** (z. B. Dachkanten von links nach rechts gespiegelt)
- **Vertikales Spiegeln**
- **Zufällige 90°-Rotation** (0°, 90°, 180° oder 270°)
- **Helligkeit und Kontrast**: zufällige Änderung innerhalb von ±10 %
- **Farbverschiebung (HSV)**:
  - **Hue (Farbton)**: ±10
  - **Saturation (Sättigung)**: ±10
  - **Value (Helligkeit)**: ±10

Die jeweilige Kombination wurde pro Augmentierungsdurchlauf neu gewählt, sodass **unterschiedliche Transformationen je Bild** möglich sind. Dieser Zufallsfaktor erhöht die Datenvielfalt und minimiert die Gefahr von Overfitting.

### Segmentierungs-Masken

Die zugehörigen **Segmentierungsmasken wurden synchron mit den Bildern transformiert**, um die Konsistenz zwischen Bild und Annotation zu erhalten. Dadurch bleiben die semantischen Informationen trotz visueller Veränderung vollständig erhalten.

### Ausgabeformat

Die augmentierten Bilder und ihre Annotations wurden im Anschluss gemeinsam mit den Originaldaten in einem **neuen COCO-kompatiblen Datensatz** gespeichert. Dieser kann nahtlos für Trainingszwecke in Frameworks wie Detectron2, MMDetection oder YOLOv8 weiterverwendet werden.

---

## ⚙️ Erklärung der Augmentierungsparameter

| Variable             | Beschreibung                                                         |
|----------------------|----------------------------------------------------------------------|
| `AUGS_PRO_BILD`      | Anzahl augmentierter Bilder, die pro Original erzeugt werden (z. B. 2) |
| `BRIGHTNESS_LIMIT`   | Max. relative Helligkeitsänderung, z. B. 0.1 = ±10 %                 |
| `CONTRAST_LIMIT`     | Max. relative Kontraständerung, analog zu `BRIGHTNESS_LIMIT`         |
| `HUE_SHIFT`          | Max. Verschiebung des Farbtons (z. B. ±10 in HSV-Farbraum)           |
| `SAT_SHIFT`          | Max. Änderung der Farbsättigung (z. B. ±10)                          |
| `VAL_SHIFT`          | Max. Änderung der Helligkeit im HSV-Modell                           |
| `p` bei jedem Schritt| Wahrscheinlichkeit, mit der dieser Schritt ausgeführt wird          |



In [ ]:
import albumentations as A
import cv2
import json
import os
import numpy as np
import shutil
from pycocotools import mask as mask_utils
from tqdm import tqdm
import random

# === 📋 EINSTELLUNGEN ===

# Wie viele augmentierte Bilder pro Originalbild erzeugen?
AUGS_PRO_BILD = 2

# Maximale Veränderung (±) bei Helligkeit & Kontrast in Prozent (0.1 = ±10%)
BRIGHTNESS_LIMIT = 0.1
CONTRAST_LIMIT = 0.1

# Farbverschiebung (max. Veränderung in absoluten Werten für H, S, V)
HUE_SHIFT = 10         # max ±10 Farbton
SAT_SHIFT = 10         # max ±10 Sättigung
VAL_SHIFT = 10         # max ±10 Helligkeit

# === 🔁 AUGMENTIERUNGEN MIT WAHRSCHEINLICHKEITEN ===
AUGMENTATIONS = [
    ("flipH", A.HorizontalFlip(p=0.5)),
    ("flipV", A.VerticalFlip(p=0.5)),
    ("rot90", A.RandomRotate90(p=0.3)),
    ("brightness", A.RandomBrightnessContrast(
        brightness_limit=BRIGHTNESS_LIMIT,
        contrast_limit=CONTRAST_LIMIT,
        p=0.4)),
    ("hue", A.HueSaturationValue(
        hue_shift_limit=HUE_SHIFT,
        sat_shift_limit=SAT_SHIFT,
        val_shift_limit=VAL_SHIFT,
        p=0.2))
]

# === 📂 VERZEICHNISSE ===
source_dir = "datensatz"
target_dir = "datensatz_augmented"
os.makedirs(target_dir, exist_ok=True)

# === 📖 JSON LADEN ===
with open(os.path.join(source_dir, "annotations.json"), "r") as f:
    coco_data = json.load(f)

# Neue COCO-Struktur
extended_data = {
    "images": [],
    "annotations": [],
    "categories": coco_data["categories"]
}

# Originalbilder kopieren
for img_entry in coco_data["images"]:
    src_path = os.path.join(source_dir, img_entry["file_name"])
    dst_path = os.path.join(target_dir, img_entry["file_name"])
    if not os.path.exists(dst_path):
        shutil.copy(src_path, dst_path)
    extended_data["images"].append(img_entry)

for ann in coco_data["annotations"]:
    extended_data["annotations"].append(ann)

# ID-Offsets berechnen
next_image_id = max(img["id"] for img in coco_data["images"]) + 1
next_ann_id = max(ann["id"] for ann in coco_data["annotations"]) + 1

# Bild-ID zu Annotationen mappen
img_to_anns = {}
for ann in coco_data["annotations"]:
    img_to_anns.setdefault(ann["image_id"], []).append(ann)

# === 🚀 AUGMENTIERUNG ===
for img_entry in tqdm(coco_data["images"], desc="Augmentiere Bilder"):
    file_name = img_entry["file_name"]
    img_path = os.path.join(source_dir, file_name)
    img = cv2.imread(img_path)
    if img is None:
        print(f"⚠️ Bild nicht lesbar: {file_name}")
        continue
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    anns = img_to_anns.get(img_entry["id"], [])
    if not anns:
        continue

    # Maske generieren
    masks = []
    for ann in anns:
        for seg in ann["segmentation"]:
            pts = np.array(seg).reshape(-1, 2)
            mask = np.zeros(img.shape[:2], dtype=np.uint8)
            cv2.fillPoly(mask, [pts.astype(np.int32)], 1)
            masks.append(mask)

    for aug_nr in range(AUGS_PRO_BILD):
        selected_transforms = []
        aug_names = []

        for name, aug in AUGMENTATIONS:
            if random.random() < aug.p:  # Zufällige Auswahl auf Basis der Wahrscheinlichkeit
                selected_transforms.append(aug)
                aug_names.append(name)

        if not selected_transforms:
            continue  # Wenn keine Transformation ausgewählt wurde, überspringen

        composed = A.Compose(selected_transforms)
        transformed = composed(image=img, masks=masks)
        aug_img = transformed["image"]
        aug_masks = transformed["masks"]

        new_filename = f"{'_'.join(aug_names)}_{aug_nr}_{file_name}"
        new_img_path = os.path.join(target_dir, new_filename)
        cv2.imwrite(new_img_path, cv2.cvtColor(aug_img, cv2.COLOR_RGB2BGR))

        extended_data["images"].append({
            "id": next_image_id,
            "file_name": new_filename,
            "width": aug_img.shape[1],
            "height": aug_img.shape[0]
        })

        for mask in aug_masks:
            rle = mask_utils.encode(np.asfortranarray(mask.astype(np.uint8)))
            area = mask_utils.area(rle).item()
            bbox = mask_utils.toBbox(rle).tolist()

            seg = mask_utils.encode(np.asfortranarray(mask))
            seg["counts"] = seg["counts"].decode("utf-8")

            extended_data["annotations"].append({
                "id": next_ann_id,
                "image_id": next_image_id,
                "category_id": 1,
                "segmentation": seg,
                "area": area,
                "bbox": bbox,
                "iscrowd": 0
            })
            next_ann_id += 1

        next_image_id += 1

# === 💾 SPEICHERN ===
json_out = os.path.join(target_dir, "annotations_augmented.json")
with open(json_out, "w") as f:
    json.dump(extended_data, f)

print(f"\n✅ Augmentierung abgeschlossen.")
print(f"📂 Neue Bilder liegen in: {target_dir}")
print(f"📄 Neue COCO-Annotations unter: {json_out}")


## Visualisierung der Augmentation

In [ ]:
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import numpy as np
import os
from IPython.display import display
from pycocotools import mask as mask_utils

# Pfade
json_path = f"{target_dir}/annotations_augmented.json"
img_dir = target_dir

# COCO laden
coco = COCO(json_path)
images = coco.dataset["images"]

# 📌 Nur augmentierte Bilder auswählen (z. B. "flipH_", "flipV_", "rot90_")
aug_images = [img for img in images if img["file_name"].startswith(("flip", "rot", "flipH", "flipV"))]

# Erstes augmentiertes Bild
img_info = aug_images[0]
img_path = os.path.join(img_dir, img_info["file_name"])
image = Image.open(img_path)

# Masken laden
ann_ids = coco.getAnnIds(imgIds=img_info["id"])
anns = coco.loadAnns(ann_ids)

# Plot
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(image)

for ann in anns:
    seg = ann["segmentation"]
    
    if isinstance(seg, list):  # Polygon
        for s in seg:
            poly = np.array(s).reshape((len(s) // 2, 2))
            patch = patches.Polygon(poly, fill=False, edgecolor='red', linewidth=2)
            ax.add_patch(patch)
    elif isinstance(seg, dict) and "counts" in seg:  # RLE
        rle = {
            "counts": seg["counts"].encode("utf-8"),
            "size": seg["size"]
        }
        mask = mask_utils.decode(rle)
        ax.contour(mask, colors='red', linewidths=2)

ax.set_title(f"Augmentiertes Bild: {img_info['file_name']}")
ax.axis("off")
display(fig)
plt.close(fig)

## Datensatz prüfen

In [ ]:
import os
import json


# 📖 JSON laden
with open(json_path, "r") as f:
    data = json.load(f)

# 🔎 Bildnamen aus JSON
json_images = {img["file_name"] for img in data["images"]}
json_image_ids = {img["id"] for img in data["images"]}

# 🔎 Bilddateien im Ordner
folder_images = {f for f in os.listdir(img_dir) if f.lower().endswith((".jpg", ".png"))}

# 🔎 IDs aus Annotations
referenced_ids = {ann["image_id"] for ann in data["annotations"]}

# ✅ Checks
missing_files = json_images - folder_images
unreferenced_files = folder_images - json_images
annotations_without_images = referenced_ids - json_image_ids

# 🖨️ Ergebnis
print("📁 Bilder in JSON, aber nicht im Ordner:", missing_files if missing_files else "✅ Keine")
print("📁 Bilder im Ordner, aber nicht in JSON:", unreferenced_files if unreferenced_files else "✅ Keine")
print("📛 Annotations mit fehlenden Bild-IDs:", annotations_without_images if annotations_without_images else "✅ Keine")


## Datensatz Hochladen

In [ ]:
import os
import requests
from getpass import getpass
from tqdm import tqdm

# 🔢 Gruppennummer eintragen
gruppen_nummer = "60"  # z. B. "01", "17", "60"

# 📂 Lokaler Ordner mit den exportierten JSONs/Bildern
target_dir = f"datensatz_augmented"

# 🌐 Ziel: annotierte_daten innerhalb der Gruppenstruktur
webdav_base = f"https://mrzinken.duckdns.org/remote.php/dav/files/hackathon2025/HackathonBonn/Gruppe{gruppen_nummer}/annotierte_daten"

# 🔐 Zugangsdaten
username = "hackathon2025"
password = "upload2025"

# 📤 Upload-Schleife
for fname in tqdm(os.listdir(target_dir), desc="📤 Upload läuft"):
    fpath = os.path.join(target_dir, fname)
    if os.path.isfile(fpath):
        remote_url = f"{webdav_base}/{fname}"
        with open(fpath, "rb") as f:
            r = requests.put(remote_url, data=f, auth=(username, password))
        if r.status_code in [200, 201, 204]:
            print(f"✅ Hochgeladen: {fname}")
        else:
            print(f"❌ Fehler bei {fname}: {r.status_code} {r.text}")

print("🏁 Upload abgeschlossen.")
